In [ ]:
#Preprocessing of input data to be used for creation of gridded product OSnet
#date : February 2022
#author : Etienne Pauthenet (etienne.pauthenet@gmail.com)

import datetime as dt
import glob
import netCDF4 as nc
import numpy as np
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from gsw import sigma0

os.getcwd()

In [ ]:
#Put the MDT, bathy etc on the SLA grid
#Create the grid of SLA
path = '/home/datawork-lops-bluecloud/osnet/data_remote_sensing/SLA/SLA_Gulf_Stream/'
ds_sla = xr.open_mfdataset(path + 'SLA_Gulf_Stream_*.nc',combine='nested', concat_dim='time')
ds_sla = ds_sla.assign_coords(lon180=(((ds_sla.longitude + 180) % 360) - 180))  
ds_sla['longitude'] = ds_sla.lon180

ds_input = xr.Dataset({'lat': (['lat'], ds_sla.latitude),
                     'lon': (['lon'], ds_sla.longitude),
                    'time':(['time'], ds_sla.time)})

sla   = ds_sla.interp(latitude=ds_input.lat,longitude=ds_input.lon,method = 'linear')['sla'].astype(np.float32).values
ugos  = ds_sla.interp(latitude=ds_input.lat,longitude=ds_input.lon,method = 'linear')['ugos'].astype(np.float32).values
vgos  = ds_sla.interp(latitude=ds_input.lat,longitude=ds_input.lon,method = 'linear')['vgos'].astype(np.float32).values
ugosa = ds_sla.interp(latitude=ds_input.lat,longitude=ds_input.lon,method = 'linear')['ugosa'].astype(np.float32).values
vgosa = ds_sla.interp(latitude=ds_input.lat,longitude=ds_input.lon,method = 'linear')['vgosa'].astype(np.float32).values
sla_err = ds_sla.interp(latitude=ds_input.lat,longitude=ds_input.lon,method = 'linear')['err'].astype(np.float32).values

ds_input = ds_input.assign(variables={"SLA": (("time","lat","lon"),sla)})  
ds_input = ds_input.assign(variables={"UGOS": (("time","lat","lon"),ugos)})  
ds_input = ds_input.assign(variables={"VGOS": (("time","lat","lon"),vgos)})  
ds_input = ds_input.assign(variables={"UGOSA": (("time","lat","lon"),ugosa)})  
ds_input = ds_input.assign(variables={"VGOSA": (("time","lat","lon"),vgosa)})
ds_input = ds_input.assign(variables={"SLA_err": (("time","lat","lon"),sla_err)})
del ds_sla,sla,ugos,vgos,ugosa,vgosa,sla_err

In [ ]:
#Load Bathy
ds_bat = xr.open_dataset('/home/datawork-lops-bluecloud/osnet/bathymetry_GulfStream.nc')
ds_bat = ds_bat.expand_dims("TIME")
bat = ds_bat.interp(LATITUDE=ds_input.lat,LONGITUDE=ds_input.lon,method = 'linear')['bathymetry'].astype(np.float32).squeeze().values
#bat_rep = np.repeat(bat,len(ds_input.time),axis = 0)
ds_input = ds_input.assign(variables={"BATHY": (("lat","lon"),bat)})  
del ds_bat,bat

In [ ]:
#Load MDT and subsample
path = '/home/datawork-lops-bluecloud/osnet/data_remote_sensing/MDT/'
ds = xr.open_dataset(path + 'mdt-cnes-cls18-global.nc')
ds_mdt = ds.where((ds.longitude<=-25+360) & (ds.longitude>=-85+360) & (ds.latitude<=55) & (ds.latitude>=18), drop=True)
ds_mdt = ds_mdt.assign_coords(lon180=(((ds_mdt.longitude + 180) % 360) - 180))  
ds_mdt['longitude'] = ds_mdt.lon180  

mdt = ds_mdt.interp(latitude=ds_input.lat,longitude=ds_input.lon,method = 'linear')['mdt'].astype(np.float32).squeeze().values
#mdt_rep = np.repeat(mdt,len(ds_input.time),axis =0)
ds_input = ds_input.assign(variables={"MDT": (("lat","lon"),mdt)})  
del ds_mdt,mdt

In [ ]:
%%time
#Load SST and subsample
path = '/home/datawork-lops-bluecloud/osnet/data_remote_sensing/SST/SST_Gulf_Stream/'
ds_sst = xr.open_mfdataset(path + '*.nc',combine='nested', concat_dim='time')

sst = ds_sst.interp(lat=ds_input.lat,lon=ds_input.lon,method = 'linear')['analysed_sst'].values
sstE = ds_sst.interp(lat=ds_input.lat,lon=ds_input.lon,method = 'linear')['analysis_uncertainty'].values

ds_input = ds_input.assign(variables={"SST": (("time","lat","lon"),sst-273.15)})  
ds_input = ds_input.assign(variables={"SST_uncertainty": (("time","lat","lon"),sstE)})
del ds_sst,sst,sstE

ds_input.to_netcdf('Gridded_input_intermediate.nc')
%reset

In [ ]:
ds_input = xr.open_dataset('Gridded_input_intermediate.nc')

#Remove gridpoints with bathy<1000
ds_input = ds_input.where(ds_input.BATHY < -1000)

#Apply the NaN mask
mask_ocean = 1 * np.ones((ds_input.dims['lat'], ds_input.dims['lon'])) * np.isfinite(ds_input.SST.isel(time=0))  
mask_land = 0 * np.ones((ds_input.dims['lat'], ds_input.dims['lon'])) * np.isnan(ds_input.SST.isel(time=0))  
mask_array = mask_ocean + mask_land
ds_input.coords['mask'] = (('lat', 'lon'), mask_array)
ds_input = ds_input.where(ds_input['mask'] == 1)

In [ ]:
#Remove the border that do not correspond between surface datasets
ds_input = ds_input.where((ds_input.lon<=-30) & (ds_input.lon>=-80) & (ds_input.lat<=50) & (ds_input.lat>=23), drop=True)

#Verification that each gridpoint is either NaN or value (0 or 5)
mask_SST = 1 * np.ones((ds_input.dims['lat'], ds_input.dims['lon'])) * np.isfinite(ds_input.SST.isel(time=0))  
mask_SLA = 1 * np.ones((ds_input.dims['lat'], ds_input.dims['lon'])) * np.isfinite(ds_input.SLA.isel(time=0))  
mask_MDT = 1 * np.ones((ds_input.dims['lat'], ds_input.dims['lon'])) * np.isfinite(ds_input.MDT)  
mask_BAT = 1 * np.ones((ds_input.dims['lat'], ds_input.dims['lon'])) * np.isfinite(ds_input.BATHY)  
mask_sum = mask_SST + mask_SLA + mask_MDT + mask_BAT
mask_sum.plot()
print(set(np.array(mask_sum).flatten()))

In [ ]:
#Final save
ds_input.to_netcdf('Gridded_input.nc')